# Self Subduction Case 2

In [ ]:
import UWGeodynamics as GEO
import numpy as np
from UWGeodynamics import visualisation as vis

In [ ]:
u = GEO.UnitRegistry

In [ ]:
half_rate = 1.0 * u.centimeter / u.year
model_length = 3000. * u.kilometer
surfaceTemp = 273.15 * u.degK
baseModelTemp = 1603.15 * u.degK
bodyforce = 3300 * u.kilogram / u.metre**3 * 9.81 * u.meter / u.second**2

KL = model_length
Kt = KL / half_rate
KM = bodyforce * KL**2 * Kt**2
KT = (baseModelTemp - surfaceTemp)

GEO.scaling_coefficients["[length]"] = KL
GEO.scaling_coefficients["[time]"] = Kt
GEO.scaling_coefficients["[mass]"]= KM
GEO.scaling_coefficients["[temperature]"] = KT

In [ ]:
Model = GEO.Model(elementRes=(256, 128), 
                  minCoord=(-1500. * u.kilometer, -700. * u.kilometer), 
                  maxCoord=(1500. * u.kilometer, 50. * u.kilometer), 
                  gravity=(0.0, -9.81 * u.meter / u.second**2))

In [ ]:
angle = 34.0 * u.degree
dx1 = np.cos(angle) * 375. * u.kilometer
dy1 = np.sin(angle) * 375. * u.kilometer 
dx2 = np.sin(angle) * 100. * u.kilometer
dy2 = np.cos(angle) * 100. * u.kilometer

In [ ]:
polygon1 = GEO.shapes.Polygon([(-500. * u.kilometer, 0. * u.kilometer),
                              (-500. * u.kilometer - dx1, 0. * u.kilometer - dy1),
                              (-500. * u.kilometer - dx1 + dx2, 0. * u.kilometer - dy1 - dy2),
                              (-500. * u.kilometer + dx2, 0. * u.kilometer - dy2)])

polygon2 = GEO.shapes.Polygon([(-500. * u.kilometer, 0. * u.kilometer),
                              (-500. * u.kilometer, -100. * u.kilometer),
                              (1500. * u.kilometer , -100. * u.kilometer),
                              (1500. * u.kilometer, 0. * u.kilometer)])

polygon = polygon1 + polygon2

In [ ]:
stickyAir = Model.add_material(name="Mantle", shape=GEO.shapes.Layer(top=Model.top, bottom=0. * u.kilometer))
Mantle = Model.add_material(name="Mantle", shape=GEO.shapes.Layer(top=0.*u.kilometer, bottom=Model.bottom))
Lithosphere = Model.add_material(name="Lithosphere", shape=polygon)

In [ ]:
Fig = vis.Figure(figsize=(1200,400))
Fig.Points(Model.swarm, Model.materialField)
Fig.show()

In [ ]:
tip_tracer1 = Model.add_passive_tracers(name="tip tracer 1", vertices=[GEO.nd(-500. * u.kilometer - dx1), GEO.nd(0. * u.kilometer - dy1)])
tip_tracer2 = Model.add_passive_tracers(name="tip tracer 2", vertices=[GEO.nd(-500. * u.kilometer - dx1 + dx2/2.0), GEO.nd(0. * u.kilometer - dy1 - dy2/2.0)])
tip_tracer3 = Model.add_passive_tracers(name="tip tracer 3", vertices=[GEO.nd(-500. * u.kilometer - dx1 + dx2), GEO.nd(0. * u.kilometer - dy1 - dy2)])

In [ ]:
from UWGeodynamics import visualisation as vis
Fig = vis.Figure(figsize=(1200,400))
Fig.Points(tip_tracer1.swarm, pointSize=5.0)
Fig.Points(tip_tracer2.swarm, pointSize=5.0)
Fig.Points(tip_tracer3.swarm, pointSize=5.0)
Fig.Points(Model.swarm, Model.materialField, fn_size=3.0)
Fig.show()

In [ ]:
stickyAir.viscosity = 1e19 * u.pascal * u.second
Mantle.viscosity = 1e21 * u.pascal * u.second
Lithosphere.viscosity = 1e23 * u.pascal * u.second

In [ ]:
stickyAir.density = 1. * u.kilogram / u.metre**3
Mantle.density = 3200. * u.kilogram / u.metre**3
Lithosphere.density = 3300. * u.kilogram / u.metre**3

In [ ]:
Fig = vis.Figure(figsize=(1200,400))
Fig.Points(Model.swarm, GEO.Dimensionalize(Model.densityField, u.kilogram / u.metre**3))
Fig.show()

In [ ]:
Fig = vis.Figure(figsize=(1200,400))
Fig.Points(Model.swarm, GEO.Dimensionalize(Model.viscosityField, u.pascal * u.second))
Fig.show()

In [ ]:
Model.set_velocityBCs(left=[0., None],
                      right=[0., None],
                      bottom=[None, 0.],
                      top=[None, 0.])

In [ ]:
Model.init_model()

In [ ]:
GEO.rcParams["solver"] = "mumps"
GEO.rcParams["penalty"] = 1e6

In [ ]:
Model.run_for(100. * u.megayears, checkpoint_interval=1.0 * u.megayears)